In [1]:
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [19]:
dataset = 'acdc'
labels_dict_h = {0:'drivable', 1:'flat non-drivable', 2:'obstacle', 3:'road sign', 4:'sky', 5:'human', 6:'vehicle', 7:'abstain'}
labels_dict = pickle.load(open('/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/data/cityscapes_trainIds.pkl', 'rb'))
stats = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/agg_stats_99.pkl', 'rb'))
baseline_stats = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/baseline.pkl', 'rb'))
baseline_stats_h = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/baseline_h.pkl', 'rb'))
stats_h = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/stats_h_99.pkl', 'rb'))

In [20]:
# boundary_certified_pixels_@maxsize_count_per_clas
# Certified Boundary and Non-Boundary Pixels @ different class set sizes w.r.t ground truth label
graph_dict = {}
n = 300

acc = 0
confusion_matrix = np.zeros((20, 20))
for image, n_k_d in stats.items():
    acc += baseline_stats[image]['acc']
    confusion_matrix += baseline_stats[image]['confusion_matrix']
    for n_k, d in n_k_d.items():
        n, k = n_k
        if n == 300:n=300
        else: n -= 10
        if n not in graph_dict:
            graph_dict[n] = {}
        if k not in graph_dict[n]:
            graph_dict[n][k] = {'count_certified_boundary': np.zeros(20), 
                                'count_certified_nonboundary': np.zeros(20),
                                'mean_percentage_certified': 0,
                                'mean_certified_acc_boundary': 0,
                                'mean_certified_acc_nonboundary': 0,
                                'mean_certified_acc': 0,
                                'certified_boundary_pixels_count':0,
                                'certified_nonboundary_pixels_count':0,
                                'num_boundary_pixels':0,
                                'num_nonboundary_pixels':0,
                                'strs_histo':{},
                                'certified_strs_histo': {}}
        if k != 'h':
            graph_dict[n][k]['count_certified_boundary'] += n_k_d[n_k]['boundary_certified_pixels_@maxsize_count_per_class']
            graph_dict[n][k]['count_certified_nonboundary'] += n_k_d[n_k]['nonboundary_certified_pixels_@maxsize_count_per_class']
        graph_dict[n][k]['mean_percentage_certified'] += n_k_d[n_k]['percentage_certified']
        graph_dict[n][k]['mean_certified_acc_boundary'] += n_k_d[n_k]['certified_acc_boundary']
        graph_dict[n][k]['mean_certified_acc_nonboundary'] += n_k_d[n_k]['certified_acc_nonboundary']
        graph_dict[n][k]['mean_certified_acc'] += n_k_d[n_k]['certified_acc']
        graph_dict[n][k]['certified_boundary_pixels_count'] += n_k_d[n_k]['certified_boundary_pixels_count']
        graph_dict[n][k]['certified_nonboundary_pixels_count'] += n_k_d[n_k]['certified_nonboundary_pixels_count']
        graph_dict[n][k]['num_boundary_pixels'] += n_k_d[n_k]['num_boundary_pixels']
        graph_dict[n][k]['num_nonboundary_pixels'] += n_k_d[n_k]['num_nonboundary_pixels']
        for str in n_k_d[n_k]['strs_histo']:
            if str not in graph_dict[n][k]['strs_histo']:
                graph_dict[n][k]['strs_histo'][str] = 0
            graph_dict[n][k]['strs_histo'][str] += n_k_d[n_k]['strs_histo'][str]
        for str in n_k_d[n_k]['certified_strs_histo']:
            if str not in graph_dict[n][k]['certified_strs_histo']:
                graph_dict[n][k]['certified_strs_histo'][str] = 0
            graph_dict[n][k]['certified_strs_histo'][str] += n_k_d[n_k]['certified_strs_histo'][str]

mean_baseline_acc = acc/len(stats.keys())


In [21]:
sum_h = {}

for image in stats_h.keys():
    for n in stats_h[image].keys():
        if n != 300:
            n_ = n - 10
        else:
            n_ = 300
        if n_ not in sum_h:
             sum_h[n_] = {'num_certified_pixels':0, 'num_pixels':0, 'certified_pos':0,
                'num_boundary_pixels':0, 'num_nonboundary_pixels':0,
                'num_c_boundary_pixels':0, 'num_c_nonboundary_pixels':0, 'pos_boundary_pixels':0, 'pos_nonboundary_pixels':0}
        for h in stats_h[image][n]['hierarchical_acc'].keys():
            if h not in sum_h[n_]:
                sum_h[n_][h] = {'pos': 0, 'num_h_pixels': 0, 
                        'pos_boundary_pixels':0, 'pos_nonboundary_pixels': 0,
                        'num_boundary':0, 'num_nonboundary': 0}
            sum_h[n_][h]['pos'] += stats_h[image][n]['hierarchical_acc'][h]['pos']
            sum_h[n_][h]['num_h_pixels'] += stats_h[image][n]['hierarchical_acc'][h]['num_h_pixels']
            sum_h[n_][h]['pos_boundary_pixels'] += stats_h[image][n]['hierarchical_acc'][h]['pos_boundary_pixels']
            sum_h[n_][h]['pos_nonboundary_pixels'] += stats_h[image][n]['hierarchical_acc'][h]['pos_nonboundary_pixels']
            sum_h[n_][h]['num_boundary'] += stats_h[image][n]['hierarchical_acc'][h]['num_boundary']
            sum_h[n_][h]['num_nonboundary'] += stats_h[image][n]['hierarchical_acc'][h]['num_nonboundary']
            sum_h[n_]['pos_boundary_pixels'] += stats_h[image][n]['hierarchical_acc'][h]['pos_boundary_pixels']
            sum_h[n_]['pos_nonboundary_pixels'] += stats_h[image][n]['hierarchical_acc'][h]['pos_nonboundary_pixels']

        sum_h[n_]['num_certified_pixels'] += stats_h[image][n]['num_certified_pixels']
        sum_h[n_]['num_pixels'] += stats_h[image][n]['num_pixels']
        sum_h[n_]['certified_pos'] += stats_h[image][n]['certified_pos']
        sum_h[n_]['num_boundary_pixels'] += stats_h[image][n]['num_boundary_pixels']
        sum_h[n_]['num_nonboundary_pixels'] += stats_h[image][n]['num_nonboundary_pixels']
        sum_h[n_]['num_c_boundary_pixels'] += stats_h[image][n]['num_c_boundary_pixels']
        sum_h[n_]['num_c_nonboundary_pixels'] += stats_h[image][n]['num_c_nonboundary_pixels']


graph_dict_h = {}

for n in sum_h.keys():
    n_ = n
    if n_ not in graph_dict_h:
        graph_dict_h[n_] = {}
    graph_dict_h[n_]['mean_certified_acc'] = sum_h[n]['certified_pos']/sum_h[n]['num_pixels']
    graph_dict_h[n_]['mean_pixel_certified_precision_accuracy'] = sum_h[n]['certified_pos']/sum_h[n]['num_certified_pixels']
    graph_dict_h[n_]['mean_percentage_certified'] = sum_h[n]['num_certified_pixels']/sum_h[n]['num_pixels']
    graph_dict_h[n_]['mean_percentage_certified_boundary'] = sum_h[n]['num_c_boundary_pixels']/sum_h[n]['num_boundary_pixels']
    graph_dict_h[n_]['mean_percentage_certified_nonboundary'] = sum_h[n]['num_c_nonboundary_pixels']/sum_h[n]['num_nonboundary_pixels']
    graph_dict_h[n_]['mean_certified_acc_boundary'] = sum_h[n]['pos_boundary_pixels']/sum_h[n]['num_boundary_pixels']
    graph_dict_h[n_]['mean_certified_acc_nonboundary'] = sum_h[n]['pos_nonboundary_pixels']/sum_h[n]['num_nonboundary_pixels']
    graph_dict_h[n_]['certified_nonboundary_pixels_count'] = sum_h[n]['num_c_nonboundary_pixels']
    graph_dict_h[n_]['certified_boundary_pixels_count'] = sum_h[n]['num_c_boundary_pixels']

    graph_dict_h[n_]['num_boundary_pixels'] = sum_h[n]['num_boundary_pixels']
    graph_dict_h[n_]['num_nonboundary_pixels'] = sum_h[n]['num_nonboundary_pixels']


In [5]:
sum_h.keys()

dict_keys([300, 200, 100, 75])

In [22]:
# baseline
baseline = {}
for h in [0, 1, 2, 3]:
    baseline[h] = {'acc':0, 'confusion_matrix': np.zeros((20, 20))}
    for image in baseline_stats_h.keys():
        baseline[h]['acc'] += baseline_stats_h[image][h]['acc']
        baseline[h]['confusion_matrix'] += baseline_stats_h[image][h]['confusion_matrix']
    baseline[h]['acc'] = baseline[h]['acc']/len(baseline_stats_h.keys())

for h in [0, 1, 2, 3]:
    print(h+1, baseline[h]['acc'])

1 0.5069139901587084
2 0.6081753605326845
3 0.6482189577763017
4 0.8463971015853075


In [23]:

pos = confusion_matrix.sum(1)
res = confusion_matrix.sum(0)
tp = np.diag(confusion_matrix)
pixel_acc = tp.sum()/pos.sum()
print('pixel_acc', pixel_acc)

pixel_acc 0.5853813608304627


In [24]:
graph_dict_h.keys()

dict_keys([300, 200, 100, 75])

In [25]:
options = ['mean_percentage_certified', 'mean_certified_acc']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

i = 1
fig = make_subplots(rows=1, cols=2, subplot_titles=options, shared_yaxes=True)
show_legend = True
for option in options:
    k_n = {}
    x = []
    y_h = []
    for n, n_d in graph_dict.items():
        for k, v in n_d.items():
            if k not in k_n:
                k_n[k] = []
            k_n[k].append(graph_dict[n][k][option]/len(stats.keys()))
        x.append(n)
        y_h.append(graph_dict_h[n][option])
    for j, k in enumerate(k_n.keys()):
        fig.add_scatter(x=list(reversed(x)), y=list(reversed(k_n[k])), name=f'set_size={k}', row=1, col=i, showlegend=show_legend, line_color=colors[j])
        fig.update_xaxes(row=1, col=i, title_text='Number of Samples', )
        fig.update_yaxes(row=1, col=i, title_text= f'{option}' )
    fig.add_scatter(x=list(reversed(x)), y=list(reversed(y_h)), name=f'relaxed hierarchical', row=1, col=i, showlegend=show_legend, line_color='black', line = dict(color='black', width=2))

    #fig.add_scatter(x=list(reversed(x)), y=[mean_baseline_acc]*len(list(reversed(k_n[k]))), name=f'baseline accuracy', row=1, col=i, showlegend=show_legend, line_color='gray', line = dict(color='gray', width=2, dash='dash'))

    show_legend = not show_legend
    i+=1

fig.update_layout(
    title=f'Mean Percentage Certified Pixels and Mean Certified Accuracy ({len(stats.keys())} {dataset} Validation Samples)',
)
fig.update_traces(overwrite=True)
fig.show()



In [26]:
options = ['mean_certified_acc_nonboundary', 'mean_certified_acc_boundary']


i = 1
fig = make_subplots(rows=1, cols=2, subplot_titles=options, shared_yaxes=True)
show_legend = True
for option in options:
    k_n = {}
    x = []
    y_h = []
    for n, n_d in graph_dict.items():
        for k, v in n_d.items():
            if k not in k_n:
                k_n[k] = []
            k_n[k].append(graph_dict[n][k][option]/len(stats.keys()))
        x.append(n)
        y_h.append(graph_dict_h[n][option])
    for j, k in enumerate(k_n.keys()):
        fig.add_scatter(x=list(reversed(x)), y=list(reversed(k_n[k])), name=f'set_size={k}', row=1, col=i, showlegend=show_legend, line_color=colors[j])
        fig.update_xaxes(row=1, col=i, title_text='Number of Samples', )
        fig.update_yaxes(row=1, col=i, title_text= f'{option}' )
    #fig.add_scatter(x=list(reversed(x)), y=[mean_baseline_acc]*len(list(reversed(k_n[k]))), name=f'baseline', row=1, col=i, showlegend=show_legend, line_color='black', line = dict(color='black', width=2, dash='dash'))
    fig.add_scatter(x=list(reversed(x)), y=list(reversed(y_h)), name=f'relaxed hierarchical', row=1, col=i, showlegend=show_legend, line_color='black')

    show_legend = not show_legend
    i+=1

fig.update_layout(
    title=f'Mean Certified Accuracy ({len(stats.keys())} {dataset} Validation Samples)',
)
fig.update_traces(overwrite=True)
fig.show()

In [27]:
options = ['certified_nonboundary_pixels_count', 'certified_boundary_pixels_count']
denom = ['num_nonboundary_pixels', 'num_boundary_pixels']
types_ = ['nonboundary', 'boundary']

i = 1
fig = make_subplots(rows=1, cols=2, subplot_titles=types_, shared_yaxes=True)
show_legend = True
for o_i, option in enumerate(options):
    k_n = {}
    x = []
    y_h = []
    for n, n_d in graph_dict.items():
        for k, v in n_d.items():
            if k not in k_n:
                k_n[k] = []
            k_n[k].append(graph_dict[n][k][option]/graph_dict[n][k][denom[o_i]])
        x.append(n)
        y_h.append(graph_dict_h[n][option]/graph_dict_h[n][denom[o_i]])
    for j, k in enumerate(k_n.keys()):
        fig.add_scatter(x=list(reversed(x)), y=list(reversed(k_n[k])), name=f'set_size={k}', row=1, col=i, showlegend=show_legend, line_color=colors[j])
        fig.update_xaxes(row=1, col=i, title_text='Number of Samples', )
        fig.update_yaxes(row=1, col=i, title_text= f'{option}' )
    fig.add_scatter(x=list(reversed(x)), y=list(reversed(y_h)), name=f'relaxed hierarchical', row=1, col=i, showlegend=show_legend, line_color='black')

    show_legend = not show_legend
    i+=1

fig.update_layout(
    title=f'Percentage of Certified Pixels ({len(stats.keys())} {dataset} Validation Samples)',
)
fig.update_traces(overwrite=True)
fig.show()

In [12]:
sum_h.keys()

dict_keys([300, 200, 100, 75])

In [28]:
n_ = 75
x = []
options = ['pos', 'neg']
denom = ['num_nonboundary_pixels', 'num_boundary_pixels']
traces = ['correctly certified', 'wrongly certified']
colors = ['purple', 'blue', 'red', 'green']
fig = go.Figure()
show_legend = True
o_i = 0
for option in options:
    x, y = [], []
    for h in sum_h[n_].keys():
        if type(h) == int:
            x.append(f'h={h+1}')
            if option=='neg':
                y.append(sum_h[n_][h]['num_h_pixels'] - sum_h[n_][h]['pos'])
            else:
                y.append(sum_h[n_][h]['pos'])
    print(x, y)
    fig.add_bar(x=x, y=y, name=f'{traces[o_i]}')
    fig.update_xaxes(title_text='Hierarchy level: fine (19) to coarse (3)', )
    fig.update_yaxes(title_text= f'Number of certified pixels' )
    o_i += 1

fig.update_layout(
    title=f'Certified pixels count at different hierarchies in the relaxed hierarchy approach',
    barmode='stack'
)
fig.update_traces(overwrite=True)
fig.show()

['h=1', 'h=2', 'h=3', 'h=4'] [30826853, 2891380, 1888935, 3056755]
['h=1', 'h=2', 'h=3', 'h=4'] [4456371, 287838, 150371, 210674]


In [29]:
n_ = 75
x = []
options = ['certified_nonboundary_pixels_count', 'certified_boundary_pixels_count']
denom = ['num_nonboundary_pixels', 'num_boundary_pixels']
traces = ['nonboundary pixels', 'boundary pixels']
colors = ['purple', 'blue', 'red', 'green']
fig = go.Figure()
show_legend = True
i = 1
for o_i, option in enumerate(options):
    for n, n_d in graph_dict.items():
        if n == n_:
            x, y = [], []
            y_h = []
            for k, v in n_d.items():
                x.append(f'set_size={k}')
                y.append(graph_dict[n][k][option]/graph_dict[n][k][denom[o_i]])
            x.append('relaxed_hierarchical')
            y.append(graph_dict_h[n][option]/graph_dict_h[n][denom[o_i]])

            fig.add_bar(x=x, y=y, name=f'{traces[o_i]}')
            fig.update_xaxes(title_text='Set size in the greedy algorithm', )
            fig.update_yaxes(title_text= f'percentage certified pixels' )
            i += 1

fig.update_layout(
    title=f'Percentage of certified Pixels @ different algorithms variations ({len(stats.keys())} {dataset} Validation Samples)',
)
fig.update_traces(overwrite=True)
fig.show()

In [30]:
n_ = 75
x = []
options = ['certified_nonboundary_pixels_count', 'certified_boundary_pixels_count']
denom = ['num_nonboundary_pixels', 'num_boundary_pixels']
traces = ['nonboundary pixels', 'boundary pixels']
fig = go.Figure()
show_legend = True
i = 1
for o_i, option in enumerate(options):
    for n, n_d in graph_dict.items():
        if n == n_:
            x, y = [], []
            for k, v in n_d.items():
                x.append(f'set_size={k}')
                y.append(graph_dict[n][k][option]/(graph_dict[n][k][denom[0]]+graph_dict[n][k][denom[1]]))
            x.append('relaxed_hierarchical')
            y.append(graph_dict_h[n][option]/(graph_dict_h[n][denom[0]]+graph_dict_h[n][denom[1]]))
            fig.add_bar(x=x, y=y, name=f'{traces[o_i]}')
            fig.update_xaxes(title_text='Set size in the greedy algorithm', )
            fig.update_yaxes(title_text= f'percentage certified pixels' )
            i += 1

fig.update_layout(
    title=f'Percentage of certified Pixels @ different set_sizes ({len(stats.keys())} {dataset} Validation Samples)', barmode='stack'
)
fig.update_traces(overwrite=True)
fig.show()

In [31]:
n = 200
top_n = 8
pixels_type = ['nonboundary', 'boundary']
fig = make_subplots(rows=1, cols=2, subplot_titles=pixels_type, shared_yaxes=True)
show_legend = True
for i, b in enumerate(pixels_type):
    for k_i, k in enumerate(graph_dict[n].keys()):
        if k=='h': continue
        list1, y = zip(*sorted(zip(graph_dict[n][2][f'count_certified_{b}'], graph_dict[n][k][f'count_certified_{b}'])))
        list1, x = zip(*sorted(zip(graph_dict[n][2][f'count_certified_{b}'], list(labels_dict.values()))))
        x, y = list(reversed(x)), list(reversed(y))
        fig.add_bar(x=x[:top_n], y=y[:top_n], name=f'set_size={k}', row=1, col=i+1, marker_color=colors[k_i], showlegend=show_legend)
    show_legend = False
fig.update_layout(
    title=f'Count of certified pixels labels at different set sizes in the greedy algorithm (n={n})',
    xaxis_title='Class labels',
    yaxis_title = 'Pixels count'
)
fig.show()

In [32]:
n = 300
top_n = 8
fig = make_subplots(rows=1, cols=5, subplot_titles=[f'set_size={k}' for k in graph_dict[n].keys()], shared_yaxes=True)

for i, k in enumerate(graph_dict[n].keys()):
    x = list(graph_dict[n][k]['strs_histo'].keys())
    y = list(graph_dict[n][k]['strs_histo'].values())
    y, x = zip(*sorted(zip(y, x)))
    fig.add_bar(x=list(reversed(x))[:top_n], y=list(reversed(y))[:top_n], row=1, col=i+1, name=f'set_size={k}')
    fig.update_xaxes(title_text=f'{k}-class groups',  row=1, col=i+1)
    fig.update_yaxes(title_text= f'pixels count',  row=1, col=i+1 )
fig.update_layout(
    title=f'Pixels Class-Group counts vs. Class Groups by the Greedy Algorithm @ n={n}'
)
fig.show()

In [33]:
n = 300
top_n = 8
fig = make_subplots(rows=1, cols=5, subplot_titles=[f'set_size={k}' for k in graph_dict[n].keys()], shared_yaxes=False)

for i, k in enumerate(graph_dict[n].keys()):
    x = list(graph_dict[n][k]['strs_histo'].keys())
    y = list(graph_dict[n][k]['strs_histo'].values())
    y, x = zip(*sorted(zip(y, x)))
    fig.add_bar(x=list(reversed(x))[:top_n], y=list(reversed(y))[:top_n], row=1, col=i+1, name=f'set_size={k}')
    fig.update_xaxes(title_text=f'{k}-class groups',  row=1, col=i+1)
    fig.update_yaxes(title_text= f'pixels count',  row=1, col=i+1 )
fig.update_layout(
    title=f'Pixels counts vs. Class Groups by the Greedy Algorithm @ n={n}'
)
fig.show()

In [49]:
diff_stats = pickle.load(open('/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/cityscapes_logs/agg_stats/diff_stats_32', 'rb'))

In [53]:
np.logspace(0, 1, num=5)

array([ 1.        ,  1.77827941,  3.16227766,  5.62341325, 10.        ])

In [58]:
posteriors_dict = {}
for image in list(diff_stats.keys()):
    set_sizes, diff, m_std = diff_stats[image] 
    unique_sizes = np.unique(set_sizes)
    for sz in unique_sizes:
        if sz not in posteriors_dict:
            posteriors_dict[sz] = []
        posteriors_dict[sz].append(diff[set_sizes == sz].mean())

for sz in posteriors_dict.keys():
    posteriors_dict[sz] = np.array(posteriors_dict[sz]).min()

posteriors_dict

{1.0: 0.91259927,
 2.0: 0.30508864,
 3.0: 0.15650481,
 4.0: 0.08245385,
 5.0: 0.055820502,
 6.0: 0.036040578,
 7.0: 0.032427624,
 8.0: 0.008507235,
 9.0: 0.05733035,
 10.0: 0.10192262}